In [2]:
# Utilización de pandas  en el análisis de datos

In [3]:
## poder subir un archivo, en este caso .cvs ubicado en un archivo local, Le hacemos una primera mirada de los primeros y los últimos datos para ver como está compuesta, vemos información importante y si hay duplicados.


In [6]:
import pandas as pd
import os 
from collections import defaultdict


ruta_directorio = r"C:\Users\luism\.cache\kagglehub\datasets\ayeshaimran123\caffeine-collective\versions\1"
nombre_archivo = "Coffe_sales.csv" 
ruta_completa = os.path.join(ruta_directorio, nombre_archivo)

try:
    df = pd.read_csv(ruta_completa)
    print("¡Archivo cargado exitosamente!")
    print("\nPrimeras 5 filas del DataFrame:")
    print(df.head())
    print(df.tail())
    df.info()
    df.describe()
    df.duplicated().sum()

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta: {ruta_completa}")
   
except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")

Error: No se encontró el archivo en la ruta: C:\Users\luism\.cache\kagglehub\datasets\ayeshaimran123\caffeine-collective\versions\1/Coffe_sales.csv


In [ ]:


# Se le hizo un análisis al conjunto, no tiene datos faltantes o extraños
#Tenemos problemas con el formato de los tiempos


df['Full_Timestamp_Str'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)

# Para cuantificar errores
df['Transaction_DateTime'] = pd.to_datetime(df['Full_Timestamp_Str'], 
                                             errors='coerce', 
                                             format='mixed')

# Validar la limpieza y eliminar la columna temporal
nan_count = df['Transaction_DateTime'].isna().sum()
print("\n--- ¡ERROR DE TIEMPO CORREGIDO Y VALIDADO! ---")
print(f"Total de valores que fallaron en la conversión (NaT): {nan_count}")

# Limpieza del DataFrame
df = df.drop(columns=['Full_Timestamp_Str'])

# Validar la nueva estructura del DataFrame
print("\nValidación de las primeras 5 filas con el nuevo campo de tiempo:")
print(df[['Date', 'Time', 'Transaction_DateTime']].head())
df.info()

# Iterar sobre las columnas de tipo 'object' (categóricas) para ver los valores únicos
print("\n--- Inspección de Valores Únicos en Columnas Categóricas ---")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\nColumna: {col}")
    
    # Muestra los valores únicos y su frecuencia de aparición
    print(df[col].value_counts())
    
    
# Ya los datos están totalmente limpios y completos

# Ahora vamos a extraer información valiosa con ayuda de pandas

dias_ordenados = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

demanda_cruzada = pd.crosstab(
    df['Weekday'], 
    df['Time_of_Day']
).reindex(dias_ordenados)

print("\n--- Demanda Semanal por Momento del Día (Conteo de Ventas) ---")
print(demanda_cruzada)

# Agrupamos por Momento del Día y por Nombre del Café, y contamos las ventas
ventas_por_momento_y_cafe = df.groupby(['Time_of_Day', 'coffee_name']).size()

# Usamos 'idxmax' en el nivel interior (coffee_name) para encontrar el índice (nombre del café)
# que tuvo la mayor venta en cada Time_of_Day.
cafe_lider_por_momento = ventas_por_momento_y_cafe.groupby(level=0).idxmax()

print("\n--- Café Líder de Ventas en Cada Momento del Día ---")
print(cafe_lider_por_momento)

# Inicializar y generar el diccionario (Repetimos la lógica del paso anterior para asegurar la ejecución)
reportes_por_cafe = defaultdict(pd.DataFrame)

for cafe in df['coffee_name'].unique():
    # Filtrar el DataFrame original
    df_filtrado = df[df['coffee_name'] == cafe]
    
    # Crear la Tabla Dinámica (Pivot Table)
    pivot_cafe = df_filtrado.pivot_table(
        index='Time_of_Day', 
        columns='Weekday', 
        values='hour_of_day', 
        aggfunc='count'
    )
    
    # Reordenar las columnas y rellenar los posibles valores NaN (cero ventas) con 0
    pivot_cafe = pivot_cafe.reindex(columns=dias_ordenados, fill_value=0)
    
    reportes_por_cafe[cafe] = pivot_cafe


# 1. Bucle de Impresión de Todos los Reportes (La Solicitud Clave)
print("\n" + "="*50)
print("     REPORTE COMPLETO DE VENTAS POR PRODUCTO Y DEMANDA")
print("="*50)

# Iterar sobre el diccionario e imprimir cada DataFrame de la Tabla Dinámica
for nombre_cafe, reporte_df in reportes_por_cafe.items():
    print(f"\n--- ☕ REPORTE: {nombre_cafe} ---")
    print(reporte_df)
    print("-" * 50)


# Para sacar métricas seguras y poder ser certeros con los datos, debemos ver que los datos por días sean equitativos.
# Puede que hayan más lunes que jueves, llevando a lunes a tener una mayor percepción, que podría ser falsa.

# 1. Ventas Totales por Día: Contar el número de transacciones por cada día de la semana
ventas_totales_por_dia = df.groupby('Weekday').size().reindex(dias_ordenados, fill_value=0)

# 2. Conteo de Ocurrencias del Día (El paso clave para la normalización):
# Eliminamos las filas duplicadas por 'Date' y luego contamos cuántas veces aparece cada 'Weekday'.
# Esto nos dice cuántos Lunes, Martes, etc., hubo en el período total.
conteo_dias_unicos = df.drop_duplicates(subset=['Date'])['Weekday'].value_counts().reindex(dias_ordenados, fill_value=0)

# 3. Calcular la Tasa Promedio de Ventas por Día (Métrica Segura)
# Tasa Promedio = Ventas Totales / Conteo de Días Únicos
tasa_promedio_ventas = (ventas_totales_por_dia / conteo_dias_unicos).round(2).sort_values(ascending=False)

print("\n--- 📈 TASA PROMEDIO DE VENTAS DIARIAS NORMALIZADA ---")
print("Este es el verdadero indicador de demanda, libre de sesgo.")
print("-" * 50)
print(tasa_promedio_ventas)
print("-" * 50)

# Opcional: Mostrar el conteo de días únicos para verificar la normalización
print("\nConteo de Días Únicos en el Período Analizado (Para Verificación):")
print(conteo_dias_unicos)